### Merging the small files into a single one for session and creating a Dataframe where single row contains the entire session information

In [5]:
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm

In [5]:
sampled_data_file_list = glob.glob("session_data/*")
session_sampled_df = pd.read_csv(sampled_data_file_list[0])
session_sampled_df.head()

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,53_0068dc30-11ee-4417-b045-396c627ef013,1,20,t_6131a810-b1c3-4be9-92c9-54105e775151,False,False,False,True,0,0,...,0,0,1,False,7,2018-09-06,True,catalog,trackdone,endplay
1,53_0068dc30-11ee-4417-b045-396c627ef013,2,20,t_309812b2-11d2-47ed-90fb-c0dbf25b0866,False,False,False,True,1,1,...,0,0,0,False,7,2018-09-06,True,catalog,clickrow,trackdone
2,53_0068dc30-11ee-4417-b045-396c627ef013,3,20,t_34871798-7470-4bf7-85a4-e0c8e277a2d3,False,False,False,True,0,0,...,1,0,0,False,7,2018-09-06,True,catalog,trackdone,trackdone
3,53_0068dc30-11ee-4417-b045-396c627ef013,4,20,t_2c06c4b3-f2a4-4ccc-b673-b9943aa7e1e3,False,True,True,False,0,0,...,1,0,0,False,7,2018-09-06,True,catalog,trackdone,fwdbtn
4,53_0068dc30-11ee-4417-b045-396c627ef013,5,20,t_94e173bf-9244-4c4b-b9a8-2a735a3d6ccf,False,True,True,False,0,1,...,0,0,0,False,7,2018-09-06,True,catalog,fwdbtn,endplay


In [6]:
from tqdm import tqdm
sampled_batch_df = pd.DataFrame(columns = session_sampled_df.columns)
for file_name in tqdm(sampled_data_file_list):
    df = pd.read_csv(file_name)
    sampled_batch_df = sampled_batch_df.append(df)
sampled_batch_df.head()

100%|██████████| 10/10 [00:18<00:00,  2.40s/it]


,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,53_0068dc30-11ee-4417-b045-396c627ef013,1,20,t_6131a810-b1c3-4be9-92c9-54105e775151,False,False,False,True,0,0,...,0,0,1,False,7,2018-09-06,True,catalog,trackdone,endplay
1,53_0068dc30-11ee-4417-b045-396c627ef013,2,20,t_309812b2-11d2-47ed-90fb-c0dbf25b0866,False,False,False,True,1,1,...,0,0,0,False,7,2018-09-06,True,catalog,clickrow,trackdone
2,53_0068dc30-11ee-4417-b045-396c627ef013,3,20,t_34871798-7470-4bf7-85a4-e0c8e277a2d3,False,False,False,True,0,0,...,1,0,0,False,7,2018-09-06,True,catalog,trackdone,trackdone
3,53_0068dc30-11ee-4417-b045-396c627ef013,4,20,t_2c06c4b3-f2a4-4ccc-b673-b9943aa7e1e3,False,True,True,False,0,0,...,1,0,0,False,7,2018-09-06,True,catalog,trackdone,fwdbtn
4,53_0068dc30-11ee-4417-b045-396c627ef013,5,20,t_94e173bf-9244-4c4b-b9a8-2a735a3d6ccf,False,True,True,False,0,1,...,0,0,0,False,7,2018-09-06,True,catalog,fwdbtn,endplay


In [7]:
sampled_batch_df.shape

(1993200, 21)

In [8]:
sampled_batch_df.to_csv('sampled_session_data.csv', index=False)

In [7]:
session_dict = dict()
for ind in tqdm(sampled_batch_df.index):
    session_id = sampled_batch_df['session_id'][ind]
    track_id = sampled_batch_df['track_id_clean'][ind]
    not_skipped_value = 1 if sampled_batch_df['not_skipped'][ind] == True else 0
    
    session_list = session_dict.get(session_id, list())
    session_list.append(track_id)
    session_list.append(not_skipped_value)
    session_dict[session_id] = session_list

100%|██████████| 1993200/1993200 [02:02<00:00, 16284.14it/s]


In [8]:
session_filtered_df = pd.DataFrame.from_dict(session_dict, orient='index')
session_filtered_df.reset_index(inplace = True)
session_filtered_df.rename(columns={'index':'session_id'}, inplace=True)
session_filtered_df.head()

,session_id,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,53_0068dc30-11ee-4417-b045-396c627ef013,t_6131a810-b1c3-4be9-92c9-54105e775151,1,t_309812b2-11d2-47ed-90fb-c0dbf25b0866,1,t_34871798-7470-4bf7-85a4-e0c8e277a2d3,1,t_2c06c4b3-f2a4-4ccc-b673-b9943aa7e1e3,0,t_94e173bf-9244-4c4b-b9a8-2a735a3d6ccf,...,t_7c13b9d7-39b7-48e0-828f-6c920ac6535f,1,t_17ee7a20-975e-4785-8e31-10d0da37457b,0,t_6e6b433c-59da-49bc-9bdf-ddc122928277,1,t_6e6b433c-59da-49bc-9bdf-ddc122928277,0,t_309812b2-11d2-47ed-90fb-c0dbf25b0866,0
1,53_006d0a57-11b2-42dd-98b6-625bb28283f6,t_8c1a8a18-7181-4364-bca4-94b97e5dc2d8,0,t_665cc6c4-7745-47bc-bb65-11eeb33e1ecc,0,t_a646aa9c-0c00-45a5-aa53-cb428e171251,1,t_69ef057b-3e0c-469e-a71a-4e348f6b6aed,0,t_2d9aef46-0e33-479f-ace4-019793ad5ea5,...,t_33831664-66b6-4590-9939-052d3c8c4b11,0,t_13ceba8d-4d30-44c7-a80d-9f75adca2432,0,t_6966e4af-fbdc-4e95-ab49-6af31c95f37a,0,t_26eff975-cb7e-4507-a3b6-278b526584bc,0,t_68613f0d-1617-4bd6-aacb-372a79ddd770,1
2,53_038a6aaa-7d85-4bab-bd25-2f69a3eb12b6,t_6838642c-b80e-4b0b-b18f-5389fb74d00f,0,t_d1f175e8-a157-411c-9447-d48615e71ed0,0,t_da1fef49-47b5-4236-842a-36347cb78460,0,t_e000b439-a7e6-4e85-84b7-4e3d54742d60,0,t_d55fd565-fe40-4442-b8f0-393e9222d23e,...,t_88e8671d-e6ee-48dc-b610-6d1cad2657cf,1,t_b7954b03-6717-42f4-a00d-fcaa26ac7f90,1,t_0337a921-6153-4c8b-8c27-679a3796dfc8,1,t_93cccc0a-8dcb-4697-b501-dc9056309abc,1,t_bf81cf32-8335-457d-a867-5fbaca8d56b9,1
3,53_040eeedd-a7ce-4229-a1c0-17e510ea9f76,t_d9b73bdb-eecf-4c8c-9f85-c2bd3bab8d33,0,t_4c316d3c-ab2e-4943-b0cd-c59a0c7480b4,0,t_d9b73bdb-eecf-4c8c-9f85-c2bd3bab8d33,0,t_59647f23-dd08-45d1-8eeb-03fdd9c1b718,0,t_ad62139c-561c-47d9-9481-744b570d74e7,...,t_d1f58de5-01c0-4bda-962c-62fed400e2f4,0,t_18b618d9-ad3e-4d24-8e6e-8e3c2c502ded,0,t_0db93ef0-06e8-460a-852a-9cf2752ae224,1,t_675dd421-0c06-4914-8beb-329eff278481,0,t_ac6deb7f-8a35-4db3-840f-4f06a7a1aab0,1
4,53_0546a593-43e6-4139-af76-bfd03c6da8fa,t_66c7eb2a-fa7e-4c56-ad55-3ee477cba71e,0,t_5704cdc3-c455-440c-868c-6a880b4c4476,0,t_6c743c12-884b-4164-b80b-895c17e700f9,1,t_3d0626fc-bf86-45e0-a61d-c49250b52771,1,t_0ba77058-5e81-4696-9bd4-c21cff3f3e82,...,t_b3c6727e-3253-4c10-a6d2-a3227f6bdc74,1,t_9d410287-c57c-4513-af4b-c0f0ea046889,1,t_8223208a-2bbf-4433-8a8c-9b8b7be92ade,1,t_01b5c742-7e2f-402f-9f54-1b4f6c04af95,1,t_821c2943-624e-4ba3-ace1-360a4c4b6e35,1


In [10]:
session_filtered_df.to_csv('session_track_reward_data.csv', index=False)

### Filtering songs based on the sampled sessions

In [9]:
unique_track_id_list = sampled_batch_df.track_id_clean.unique()

In [11]:
print("Number of unique tracks in sampled data: ", len(unique_track_id_list))

Number of unique tracks in sampled data:  275328


In [12]:
track_features_data_file_list = glob.glob("track_features/*")
track_feature_df = pd.read_csv(track_features_data_file_list[0])
track_feature_df.head()

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
0,t_2e8f4b71-8a0b-4b9c-b7d8-fb5208e87f9f,326.013336,1971,99.582885,0.716209,0.366495,0.332605,0.439835,5.805774,0.238847,...,4,0.223395,0.146012,-0.706908,0.259496,0.481157,0.238427,-0.098389,-0.254960,-0.227383
1,t_dae2ec0e-ec7b-4b3e-b60c-4a884d0eccb0,147.813324,1963,97.272035,0.839460,0.362212,0.389829,0.507580,6.845427,0.420476,...,4,0.484702,0.039554,-0.539554,0.105141,0.692589,0.226047,-0.468162,0.164389,-0.769024
2,t_cf0164dd-1531-4399-bfa6-dec19cd1fedc,110.400002,1974,99.620384,0.054673,0.495002,0.589378,0.552311,9.361949,0.842938,...,4,0.818441,0.083863,-0.242108,-0.014258,0.096396,0.417641,-0.050576,-0.204757,-0.172563
3,t_0f90acc7-d5c5-4e53-901d-55610fbd090c,237.653336,1988,96.796830,0.042606,0.389634,0.359044,0.585673,6.068578,0.665398,...,4,0.594829,0.192498,0.340039,0.034846,-0.389794,0.518381,0.185008,-0.079907,-0.016978
4,t_36b9ad02-095a-443d-a697-6c7285d9410a,174.600006,1987,97.905891,0.249982,0.513640,0.485435,0.635095,7.198735,0.408715,...,4,0.591289,0.270586,-0.411061,0.165898,0.225652,0.335518,-0.036643,-0.016300,-0.446870


In [14]:
filtered_track_features_df = pd.DataFrame(columns = track_feature_df.columns)
for file_name in tqdm(track_features_data_file_list):
    df = pd.read_csv(file_name)
    filtered_track_features_df = filtered_track_features_df.append(df[df.track_id.isin(unique_track_id_list)])
filtered_track_features_df.head()

100%|██████████| 2/2 [00:32<00:00, 16.54s/it]


,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
8,t_f9b9a072-8dae-4816-bfd6-d91fc75a5744,180.133331,1989,97.720734,0.002047,0.425469,0.486543,0.654540,7.990334,0.625231,...,4,0.691534,0.188888,0.306256,-0.325102,-0.532620,0.660405,0.149624,-0.354731,0.192537
14,t_7f435320-5d3b-497d-9346-0c97429af2b9,539.760010,1980,98.383158,0.075846,0.499848,0.494943,0.461215,7.512061,0.563252,...,4,0.468440,0.180359,-0.149974,-0.185145,-0.152791,0.560996,0.047161,-0.295635,-0.074902
18,t_949441c4-4920-4eed-b929-2d67d37b7bd9,114.133331,1983,98.967866,0.002274,0.243778,0.229295,0.290262,5.105952,0.958647,...,4,0.293100,0.101301,0.186855,-0.525412,-0.293849,0.596498,0.010453,-0.280584,0.418848
21,t_63eae5ac-744a-46ee-b719-0db8e70d6f7f,164.906677,1983,99.773741,0.005024,0.338580,0.331703,0.348846,6.060782,0.886674,...,4,0.387131,0.132311,0.248023,-0.510729,-0.387413,0.602931,0.033381,-0.312206,0.365894
39,t_3c2c29b5-c738-42c0-83e1-ee0e88ac834d,182.293335,1969,99.987810,0.198284,0.336354,0.353215,0.395759,6.443376,0.338417,...,4,0.377411,0.083737,-0.306129,0.045954,0.078400,0.666640,-0.032503,-0.299952,-0.584207


In [15]:
filtered_track_features_df.shape

(275328, 30)

In [16]:
filtered_track_features_df.to_csv('sampled_track_features_data.csv', index=False)